### Identify Average Wind Shear Over Lakes in NY

This notebook follows a very similar process to "lakeTemperatureNY.ipynb". As such, documentation will not be as thorough. Many of the functions are directly copied from that notebook, so consult that notebook for details. The process for identifying wind shear is as follows:
1. Gather dataframes for surface level (10m is what's available) and at 850mb
2. Download the geography of the desired lake (using "Cayuga Lake" as usual)
3. Create a uniform grid within the lake and save the (latitude, longitude) of all grid points
4. Determine the direction of the wind (in degrees) at both 10m and 850mb levels, then take the difference at each grid point
5. Export the average of the grid points.

##### Step 0: Imports

Import required libraries

In [53]:
from herbie import Herbie
from herbie.toolbox import EasyMap

import numpy as np
import pandas as pd
import xarray as xr

import geopandas as gpd
from shapely.geometry import Point
import fiona  # used for listlayers / FileGDB access

from scipy.interpolate import griddata

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import urllib
from pathlib import Path
import requests

##### Step 1: Datetime Function

Function to allow for parameter-less input and ensure function runs smoothly. Also, localizes to UTC.

In [55]:
#Helper for getting the right date/time
def herbieDateTime(dateTime=None):
    if dateTime is None:
        dt = pd.Timestamp.utcnow().floor("h") - pd.Timedelta(hours=2)
    else:
        dt = pd.Timestamp(dateTime)

    if dt.tzinfo is not None:
        dt = dt.tz_convert("UTC").tz_localize(None)
    return dt

##### Step 2: Fetch dataframes for surface and 850mb wind features.

The dataframes fetched here are the (u,v) vectors for wind at 10m and wind at 850mb. When working with this, again, HRRR is curvilinear so the original dataframes have (y,x) pairs for each of the variables u and v. Using the pick_points() function later will reduce each of the fields "u" and "v" down to 1D.
Additional note: at 10m, the variables are "u10" and "v10" while at 850mb, the variables are "u" and "v". Blame HRRR, I guess.

In [56]:
def surfaceAnd850(dateTime=None):
    #Create the Herbie object for the corresponding date/time
    dateTime = herbieDateTime(dateTime)
    H = Herbie(
        dateTime, 
        model = "hrrr",
        product = "sfc",
        fxx = 0,
        save_dir=Path("herbie_cache"),
        overwrite=False
    )
    #Only sample the fields we need
    var_regex = r"[U|V]GRD:10 m|[U|V]GRD:850 mb"
    H.download(var_regex, verbose=False)
    #Use Regex to filter for the temperature field at the surface and 850mb
    dMixed = H.xarray(var_regex, remove_grib=True)
    d850mb = next(ds for ds in dMixed if "isobaricInhPa" in ds.coords)
    dSurface = next(ds for ds in dMixed if "heightAboveGround" in ds.coords)
    return dSurface, d850mb

##### Step 3: Fetch the lake polygon

This process is the exact same as the lake temperature notebook. Use USGS NHD, download the GeoJSON file.

In [57]:
#Fetch a lake polygon from USGS NHD and save as GeoJSON
#Lake_name should be provided as a string (e.g. "Cayuga Lake")

def fetch_lake_geojson(lake_name="Cayuga Lake"):
    overwrite=False
    timeout=60
    out_dir = Path("finger_lakes_geojson")
    out_dir.mkdir(parents=True, exist_ok=True)

    fname = lake_name.lower().replace(" ", "_") + ".geojson"
    out_path = out_dir / fname

    if out_path.exists() and not overwrite:
        return out_path

    base = "https://hydro.nationalmap.gov/arcgis/rest/services/nhd/MapServer/12/query"

    where = f"GNIS_NAME = '{lake_name}' AND FTYPE = 390"
    params = {
        "where": where,
        "outFields": "GNIS_NAME,FTYPE",
        "returnGeometry": "true",
        "f": "geojson",
        "outSR": "4326",
    }

    url = base + "?" + urllib.parse.urlencode(params)
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    geojson = r.json()

    features = geojson.get("features", [])
    if len(features) == 0:
        raise ValueError(f"No features returned for lake '{lake_name}'")
    if len(features) > 1:
        raise ValueError(
            f"Multiple features returned for lake '{lake_name}' "
            f"({len(features)} features). Refine query."
        )
    out_path.write_bytes(r.content)

    print(
        f"Saved {lake_name}: "
        f"{len(r.content):,} bytes → {out_path}"
    )

    return out_path

##### Step 4: Get the points for the grid within the lake

This step is also identical to the lake temperature notebook. Overlay the rectangular grid, pick the points within the GeoFrame.

In [58]:
def getLakeGrid(lake_name="Cayuga Lake", spacing=0.02):
    lake_geoJson = gpd.read_file(fetch_lake_geojson(lake_name))
    lakeGeom = lake_geoJson.geometry.iloc[0]
    minLon, minLat, maxLon, maxLat = lakeGeom.bounds #Define bounds for the rectangle surrounding the lake

    candidate_lons = np.arange(minLon, maxLon, spacing)
    candidate_lats = np.arange(minLat, maxLat, spacing)

    interior_points = []

    for lat in candidate_lats:
        for lon in candidate_lons:
            p = Point(lon, lat)
            if lakeGeom.contains(p):
                interior_points.append((lat, lon))
    return interior_points

##### Step 5: Find the directional shear at each (lat, lon)

This step requires using pick_point() to extract the (u,v) vectors (both are u and v are 1D fields now), and converting them into meteorological wind direction (degrees). Once the direction of each point has been determined for both the surface (10m) and 850mb, and find the difference at every (lat, lon). Finally, send the output into a GeoDataFrame.

The formula for getting the meteorological wind direction is np.degrees(np.arctan2(-u, -v)).

In [ ]:
#Find the directional shear |Surface - 850 mb|, given the lake name and the date/time.

#Fetch the temperature difference grid (Surface - 850mb), given the lake name and the date/time. Specify the spacing if desired.

def deltaWindGrid(lake_name="Cayuga Lake", dateTime=None, spacing=0.02):
    dateTime = herbieDateTime(dateTime)
    interior_points = getLakeGrid(lake_name)
    lakeSurface, lake850mb = surfaceAnd850(dateTime)
    inLats = [p[0] for p in interior_points]
    inLons = [p[1] for p in interior_points]
    #Specify the points to be passed to Herbie's pick_points() function
    points = pd.DataFrame(
        {
            "latitude" : inLats,
            "longitude" : inLons,
        }
    )
    lakeSurfaceWind = lakeSurface.herbie.pick_points(points)
    lake850Wind = lake850mb.herbie.pick_points(points)
    #Convert the u,v wind components into meteorological wind directions
    lakeSurfaceWindTheta = ((np.degrees(np.arctan2(-1 * lakeSurfaceWind["u10"].values , -1 * lakeSurfaceWind["v10"].values))) + 360)%360
    lake850WindTheta = ((np.degrees(np.arctan2(-1 * lake850Wind["u"].values , -1 * lake850Wind["v"].values))) + 360)%360
    #Get a list for the difference between the wind directions (in degrees)
    windDirectionDiff = np.abs(lakeSurfaceWindTheta - lake850WindTheta)
    windDirectionDiff = np.minimum(windDirectionDiff, 360 - windDirectionDiff)
    projectedLakePoints = gpd.GeoDataFrame(
        {
            "lake" : lake_name,
            "timestamp" : pd.Timestamp(dateTime),
            "latitude" : inLats,
            "longitude" : inLons,
            "delta_wind_shear" : windDirectionDiff,
        },
        geometry=gpd.points_from_xy(inLons, inLats),
        crs="EPSG:4326",
    )
    
    return projectedLakePoints


##### Step 6: Take the average of wind shear and return output as a Pandas dataframe

No explanation required.

In [63]:
def avgWindShear(lake_name="Cayuga Lake", dateTime = None, spacing = 0.02):
    dateTime = herbieDateTime(dateTime)
    windGrid = deltaWindGrid(lake_name, dateTime, spacing)
    if windGrid.empty:
        return pd.DataFrame(columns=["lake", "dateTime", "meanDeltaT"])
    return pd.DataFrame(
        {
            "lake": [windGrid["lake"].iloc[0]],
            "dateTime" : [windGrid["timestamp"].iloc[0]],
            "meanDeltaWindShear" : [windGrid["delta_wind_shear"].mean()],
        }
    )